Note: following these instructions require permissions to sudo as `analytics-product`, which owns the `canonical_data` database.

In [1]:
import wmfdata as wmf

Start by loading the dataset into your own database.

In [2]:
MY_DATABASE = "bearloga"

wmf.hive.load_csv(
    "countries.tsv",
    field_spec="""
        name               STRING  COMMENT 'Country name, aligned with the article on English Wikipedia',
        iso_code           STRING  COMMENT 'ISO 3166-1 two-letter country code',
        economic_region    STRING  COMMENT 'Global South/North, according to [[en:Global North and Global South]]',
        maxmind_continent  STRING  COMMENT 'Continent, according to MaxMind databases',
        is_protected       BOOLEAN COMMENT 'Whether the country appears in [[wikitech:Country_protection_list]]',
        is_eu              BOOLEAN COMMENT 'Whether the country belongs to the European Union'
    """,
    db_name=MY_DATABASE,
    table_name="countries",
    sep="\t"
)

/home/bearloga/.conda/envs/2023-02-22T18.39.24_bearloga/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Now, since it's not possible to run `sudo` commands in our Jupyter environment, in a plain SSH connection, run:

```
sudo -u analytics-product kerberos-run-command analytics-product hive -e "DROP TABLE IF EXISTS canonical_data.countries"
```

And in the same way, run the following command:

In [3]:
print(
    "sudo -u analytics-product kerberos-run-command analytics-product "
    f"hive -e 'CREATE TABLE canonical_data.countries AS SELECT * FROM {MY_DATABASE}.countries'"
)

sudo -u analytics-product kerberos-run-command analytics-product hive -e 'CREATE TABLE canonical_data.countries AS SELECT * FROM bearloga.countries'
